 import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
import pytz
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
import pytz
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [2]:
df = pd.read_csv("pm25_cleaned_dataset.csv")

In [3]:
df.shape

(28325, 2)

In [4]:
df.head()

,utc,value
0,2021-03-12T18:15:00+00:00,69.0
1,2021-03-12T17:15:00+00:00,69.0
2,2021-03-12T16:15:00+00:00,60.0
3,2021-03-12T15:15:00+00:00,58.0
4,2021-03-12T14:15:00+00:00,58.0


In [5]:
# Define the split dates
train_end = '2019-12-31'
val_end = '2020-12-31'

# Assuming your DataFrame has a column 'date' with date values
df['utc'] = pd.to_datetime(df['utc'])
df.set_index('utc', inplace=True)
df.sort_index(inplace=True)




# Split the data
y_train = df[:train_end]  # Data up to the end of 2019
y_val = df[train_end:val_end]  # Data from 2020
y_test = df[val_end:]  # Data from 2021 onward

# Print the sizes to confirm
print(f"Train size: {len(y_train)}")
print(f"Validation size: {len(y_val)}")
print(f"Test size: {len(y_test)}")

Train size: 18974
Validation size: 7859
Test size: 1518


In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disable GPU usage


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input


# Create lag features (you can change the number of lags as needed)
lag = 2
for i in range(1, lag + 1):
    df[f'lag_{i}'] = df['value'].shift(i)
df.dropna(inplace=True)  # Remove NaN values from lagging

# Split into train, validation, and test sets (replace with actual dates)
train_end = '2019-12-31'
val_end = '2020-12-31'

X_train = df[:train_end].drop('value', axis=1)  # Features for training
y_train = df[:train_end]['value']  # Target for training

X_val = df[train_end:val_end].drop('value', axis=1)  # Features for validation
y_val = df[train_end:val_end]['value']  # Target for validation

X_test = df[val_end:].drop('value', axis=1)  # Features for test
y_test = df[val_end:]['value']  # Target for test

# Normalize features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),  # Input layer
    Dense(64, activation='relu'),  # Hidden Layer 1
    Dense(32, activation='relu'),  # Hidden Layer 2
    Dense(1)  # Output Layer
])

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
history = model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), epochs=50, batch_size=32, verbose=1)

# Predictions and Evaluation
y_pred = model.predict(X_val_scaled)
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)

print(f'Validation MAE: {mae}')
print(f'Validation MSE: {mse}')

# Evaluate on the test set
y_test_pred = model.predict(X_test_scaled)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print(f'Test MAE: {test_mae}')
print(f'Test MSE: {test_mse}')



2024-12-26 19:50:05.868672: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-26 19:50:05.868698: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-12-26 19:50:05.868702: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-12-26 19:50:05.868903: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-26 19:50:05.868913: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


In [ ]:
import pandas as pd

def create_lag_features(df, lag=3):
    for i in range(1, lag + 1):
        df[f'lag_{i}'] = df[0].shift(i)
    df.dropna(inplace=True)
    return df.iloc[:, 1:], df.iloc[:, 0]  # Features (lags), Target (current)

X, y = create_lag_features(y_train, lag=3)

In [ ]:
# Rolling forecast for test set
predictions = []
history = list(y_train)  # Start with training data

for t in range(len(y_test)):
    lag_features = np.array(history[-3:]).reshape(1, -1)  # Last 3 values
    lag_features_scaled = scaler.transform(lag_features)
    
    next_prediction = model.predict(lag_features_scaled)
    next_prediction = scaler.inverse_transform(next_prediction)  # Rescale prediction
    predictions.append(next_prediction[0, 0])
    
    # Update history with actual test value
    history.append(y_test.iloc[t])

# Convert predictions to a series
predictions_series = pd.Series(predictions, index=y_test.index)